### Caminhos e Bibliotecas

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd
import gc

In [42]:
raw_zone_path = '/content/drive/My Drive/ProjetoAplicadoII/01_raw_zone/Estabelecimentos0/K3241.K03200Y0.D50913.ESTABELE'
mun_raw_zone_path = '/content/drive/My Drive/ProjetoAplicadoII/01_raw_zone/Municipios/F.K03200$Z.D50913.MUNICCSV'
#emp_raw_zone_path = '/content/drive/My Drive/ProjetoAplicadoII/01_raw_zone/Empresas0/K3241.K03200Y0.D50913.EMPRECSV'
trusted_zone_path = "/content/drive/My Drive/ProjetoAplicadoII/02_trusted_zone/bases_processadas/estabelecimento.parquet"
mun_trusted_zone_path = "/content/drive/My Drive/ProjetoAplicadoII/02_trusted_zone/bases_processadas/municipios.parquet"
#emp_trusted_zone_path = "/content/drive/My Drive/ProjetoAplicadoII/02_trusted_zone/bases_processadas/empresas.parquet"
refined_zone_path = "/content/drive/My Drive/ProjetoAplicadoII/03_refined_zone/bases_processadas/estabelecimento.parquet"

### Raw Zone

In [43]:
cols_estab = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'id_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', \
              'nome_cidade_exterior', 'pais', 'data_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_logradouro', 'logradouro', 'numero', \
              'complemento', 'bairro', 'cep', 'uf', 'municipio', 'dd1', 'telefone', 'dd2', 'telefone2', 'ddd_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_situacao_especial']
#cols_emp = ['cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificacao_responsavel', 'capital_social', 'porte_empresa', 'ente_federativo_responsavel']

In [44]:
df_0 = pd.read_csv(raw_zone_path, sep=";", names=cols_estab, dtype=str, encoding='ISO-8859-1', nrows=5_000_000)
df_mun = pd.read_csv(mun_raw_zone_path, sep=";", names=['cod_municipio', 'nome'], dtype=str, encoding='utf-8')
#df_emp = pd.read_csv(emp_raw_zone_path, sep=";", names=cols_emp, dtype=str, encoding='ISO-8859-1', nrows=1_000_000)

In [45]:
df_0.head()

,cnpj_basico,cnpj_ordem,cnpj_dv,id_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,...,municipio,dd1,telefone,dd2,telefone2,ddd_fax,fax,correio_eletronico,situacao_especial,data_situacao_especial
0,15164610,0001,06,1,LIBENTE MOVEIS PLANEJADOS,04,20231106,63,NaN,NaN,...,9221,62,39435544,62,92110630,62,30983336,contex@hotmail.com,NaN,NaN
1,15164625,0001,66,1,NaN,02,20120302,00,NaN,NaN,...,7515,44,33231254,NaN,NaN,44,33231254,NaN,NaN,NaN
2,15164640,0001,04,1,NaN,08,20180201,67,NaN,NaN,...,6377,11,40661279,NaN,NaN,NaN,NaN,fabiolnascimento2010@ig.com.br,NaN,NaN
3,15164657,0001,61,1,PREMIUM CAR,08,20150706,01,NaN,NaN,...,0957,98,36531750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15164666,0001,52,1,NaN,08,20180201,67,NaN,NaN,...,6001,21,92326623,NaN,NaN,NaN,NaN,marcoscarlos@r7.com,NaN,NaN


In [46]:
df_mun.head()

,cod_municipio,nome
0,0001,GUAJARA-MIRIM
1,0002,ALTO ALEGRE DOS PARECIS
2,0003,PORTO VELHO
3,0004,BURITIS
4,0005,JI-PARANA


### Trz Zone

In [47]:
cols_use = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'id_matriz_filial', \
            'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal_principal', \
            'tipo_logradouro', 'cep', 'uf','correio_eletronico', 'municipio', \
            'bairro', 'situacao_especial', 'data_inicio_atividade']

In [48]:
df_estabelecimento = df_0[cols_use]
df_estabelecimento['cnpj_completo'] = df_estabelecimento['cnpj_basico'] + df_estabelecimento['cnpj_ordem'] + df_estabelecimento['cnpj_dv']

/tmp/ipython-input-1399625916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estabelecimento['cnpj_completo'] = df_estabelecimento['cnpj_basico'] + df_estabelecimento['cnpj_ordem'] + df_estabelecimento['cnpj_dv']


In [49]:
df_estabelecimento.shape

(5000000, 16)

In [50]:
cols_reorder = list(df_estabelecimento.loc[:,"cnpj_basico":"cnpj_dv"]) + ["cnpj_completo"] + list(df_estabelecimento.loc[:,"id_matriz_filial":"data_inicio_atividade"])
df_estabelecimento = df_estabelecimento[cols_reorder]

In [51]:
df_estabelecimento['cnae_fiscal_principal'] = df_estabelecimento['cnae_fiscal_principal'].astype(int)
df_estabelecimento['situacao_cadastral'] = df_estabelecimento['situacao_cadastral'].astype(int)

In [52]:
# TESTES - Não Utilizado por remoção de cnae_fiscal_secundaria da tabela
def transform_cnae_secundaria():
  df_t = df_estabelecimento[df_estabelecimento['cnpj_completo'] == "15164657000161"]
  df_t['cnae_fiscal_secundaria'] = df_t['cnae_fiscal_secundaria'].str.split(',')
  df_t = df_t.explode('cnae_fiscal_secundaria')
  df_t[df_t['cnpj_completo'] == "15164657000161"]
#num_lists = df_estabelecimento["cnae_fiscal_secundaria"].apply(lambda x: isinstance(x, list)).sum()
#print(f"Valores ainda como lista: {num_lists}")

In [53]:
df_estabelecimento = df_estabelecimento[df_estabelecimento['situacao_cadastral'] == 2]

In [54]:
df_estabelecimento.shape[0]

2356648

In [55]:
df_estabelecimento.columns

Index(['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'cnpj_completo',
       'id_matriz_filial', 'nome_fantasia', 'situacao_cadastral',
       'cnae_fiscal_principal', 'tipo_logradouro', 'cep', 'uf',
       'correio_eletronico', 'municipio', 'bairro', 'situacao_especial',
       'data_inicio_atividade'],
      dtype='object')

In [56]:
df_estabelecimento.dtypes

,0
cnpj_basico,object
cnpj_ordem,object
cnpj_dv,object
cnpj_completo,object
id_matriz_filial,object
nome_fantasia,object
situacao_cadastral,int64
cnae_fiscal_principal,int64
tipo_logradouro,object
cep,object


In [57]:
df_estabelecimento.to_parquet(trusted_zone_path, index=False)
df_mun.to_parquet(mun_trusted_zone_path, index=False)

In [58]:
del df_estabelecimento
gc.collect()

155

### Ref Zone

In [59]:
df_trz = pd.read_parquet(trusted_zone_path)
df_mun_trz = pd.read_parquet(mun_trusted_zone_path)

In [60]:
df_trz = df_trz.merge(df_mun_trz, how='left', left_on='municipio', right_on='cod_municipio').drop(columns=['cod_municipio', 'municipio'])
df_trz = df_trz.rename(columns={'nome': 'municipio'})

In [61]:
df_trz = df_trz[(df_trz['uf'] == "SP") == (df_trz['municipio'] == "SP")]

In [62]:
df_trz.shape[0]

1649701

In [63]:
df_trz.to_parquet(refined_zone_path, index=False)

In [64]:
del df_trz
gc.collect()

0

In [65]:
df_ref = pd.read_parquet(refined_zone_path)

In [68]:
df_ref.head()

,cnpj_basico,cnpj_ordem,cnpj_dv,cnpj_completo,id_matriz_filial,nome_fantasia,situacao_cadastral,cnae_fiscal_principal,tipo_logradouro,cep,uf,correio_eletronico,bairro,situacao_especial,data_inicio_atividade,municipio
0,15164625,0001,66,15164625000166,1,None,2,4929902,RUA,86690000,PR,None,PARQUE RESIDENCIAL PORTAL DAS PRIMAVERAS,None,20120302,COLORADO
1,15164804,0001,01,15164804000101,1,None,2,5611203,QUADRA,73062702,DF,None,SETOR OESTE (SOBRADINHO),None,20120308,BRASILIA
2,15164857,0001,14,15164857000114,1,None,2,5611203,AVENIDA,36046000,MG,ALBCONTABILIDADE@GMAIL.COM,SANTA TEREZINHA,None,20120308,JUIZ DE FORA
3,15165045,0001,93,15165045000193,1,None,2,4543900,RUA,78255000,MT,marisadosanjos@hotmail.com,centro,None,20120308,JAURU
4,15165147,0001,09,15165147000109,1,None,2,4712100,RUA,63460000,CE,marcosmagalhaesguerra@hotmail.com,CENTRO,None,20120308,PEREIRO
